# Calculate Hours of Daylight

**Purpose of script:**

Calculating the hours of sunlight for a given latitude and day of the year
- In: Interpolated mw file to have the coordinate grid (tif)
- Out: DataFrame with calculated hours of sunlight per latitude and date (parquet)

*Note:*
- 0 means no hours of sunlight == total darkness all day long, 24 means only sunlight == total brightness all day long
- Formula taken from the following paper: Forsythe, William C., et al. "A model comparison for daylength as a function of latitude and day of year." Ecological Modelling 80.1 (1995): 87-95. --> https://www.researchgate.net/profile/Randal-Stahl/publication/216811293_A_Model_Comparison_for_Daylength_as_a_Function_of_Latitude_and_Day_of_Year/links/5b153fbca6fdcc4611e2b08f/A-Model-Comparison-for-Daylength-as-a-Function-of-Latitude-and-Day-of-Year.pdf

In [10]:
import pyproj
import pandas as pd
from math import asin, cos, atan, tan, pi, acos, sin 

import xarray

In [23]:
mw_path = r"../Data/microwave-rs/mw_interpolated/2017-01-01_mw.tif"
out_path = r"../Data/daylight_data/hours_of_daylight.parquet"

In [39]:
# Define the source and destination coordinate reference systems
src_crs = pyproj.CRS.from_epsg(3413)  # WGS84 (longitude, latitude)
dst_crs = pyproj.CRS.from_epsg(4326)  # Web Mercator (used by most online maps)

# Define the transformer object
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

df = xarray.open_dataarray(mw_path).to_dataframe().reset_index()[["y", "x"]]

# Convert all coordinates at once
lats, longs = transformer.transform(df["x"], df["y"])

df["lat"] = lats

# drop duplicated latitudes
df = df.drop_duplicates(subset="lat")
df

,y,x,lat
0,-662500.0,-636500.0,81.533891
1,-662500.0,-635500.0,81.540251
2,-662500.0,-634500.0,81.546605
3,-662500.0,-633500.0,81.552955
4,-662500.0,-632500.0,81.559299
...,...,...,...
3893301,-3324500.0,820500.0,59.128763
3893302,-3324500.0,821500.0,59.126701
3893303,-3324500.0,822500.0,59.124636
3893304,-3324500.0,823500.0,59.122570


In [40]:
df_final = pd.DataFrame()

# create a list of dates for random year since we assume the solar duration to have no significant change over the years
datelist = pd.date_range("2017-01-01", "2017-12-31")

for date in datelist[:2]:

    print(date)

    # function to calculate the hours of daylight for a given latitude and day of the year
    def calculate_hours_of_daylight(L, J):

        # 1. Predicting the revolution angle (R) from the day of the year (J). 
        R = 0.2163108 + 2 * atan(0.9671396 * tan(.00860 * (J - 186)))
        # 2. Predicting the sun's declination angle.
        P = asin(0.39795 *cos(R)) 
        # 3. Calculate parameter to differentiate between complete darkness (<= -1) and complete light (>= 1)
        param = sin((0.8333 * pi / 180) + sin(L * pi / 180) * sin(P)) / (cos(L * pi / 180) * cos(P))
        # 4. Predicting daylength (D) (plus twilight) from latitude (L) and the sun's declination angle (P).
        if param <= -1:
            D = 0
        elif param >= 1:
            D = 24
        else:
            D = 24 - (24 / pi) * acos(param)
        
        return D

    
    df_temp = df.copy()

    # get the day of the year
    J = date.dayofyear

    # calculate the hours of daylight for each latitude
    df_temp["hours_of_daylight"] = df_temp["lat"].apply(lambda x: calculate_hours_of_daylight(x, J))
    df_temp["date"] = date

    # append to final dataframe
    df_final = pd.concat([df_final, df_temp[["x", "y", "date", "hours_of_daylight"]]], axis=0)

# save as parquet
df_final.to_parquet(out_path)

2017-01-01 00:00:00
2017-01-02 00:00:00
